**Import required libraries and scripts**

In [2]:
#Import required libraries and scripts
from scripts.library_preparation import *
from scripts.utilities import *
from scripts.docking_functions import *
from scripts.clustering_functions import *
from scripts.rescoring_functions import *
from scripts.consensus_methods import *
from scripts.performance_calculation import *
from scripts.dogsitescorer import *
from scripts.get_pocket import *

[09:05:53] Initializing Normalizer
/home/alacournola/anaconda3/envs/dockm8/lib/python3.8/site-packages/Bio/pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(
/home/alacournola/anaconda3/envs/dockm8/lib/python3.8/site-packages/MDAnalysis/coordinates/chemfiles.py:108: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  MIN_CHEMFILES_VERSION = LooseVersion("0.9")
[TRJ.py:171 - <module>()] netCDF4 is not available. Writing AMBER ncdf files will be slow.
/home/alacournola/anaconda3/envs/dockm8/lib/python3.8/site-packages/MDAnalysis/coordinates/TRJ.py:1209: DeprecationWarning: Please use `netcdf_file` from the `scipy.io` namespace, the `scipy.io.netcdf` namespace is deprecated.
 

**Set up**
- **software**: The path to the software folder.
- **proteinfile**: The path to the protein file (.pdb).
- **pocket**: The method to use for pocket determination. Must be one of 'reference' or 'dogsitescorer'.
- **dockinglibrary: The path to the docking library file (.sdf).
- **idcolumn**: The unique identifier column used in the docking library.
- **protonation**: The method to use for compound protonation. Must be one of 'pkasolver', 'GypsumDL', or 'None'.
- **docking**: The method(s) to use for docking. Must be one or more of 'GNINA', 'SMINA', or 'PLANTS'.
- **metric**: The method(s) to use for pose clustering. Must be one or more of 'RMSD', 'spyRMSD', 'espsim', 'USRCAT', '3DScore', 'bestpose', 'bestpose_GNINA', 'bestpose_SMINA', or 'bestpose_PLANTS'.
- **nposes**: The number of poses to generate for each docking software. Default=10
- **exhaustiveness**: The precision used if docking with SMINA/GNINA. Default=8
- **parallel**: Whether or not to run the workflow in parallel. Default=1 (on). Can be set to 1 (on) or 0 (off).
- **ncpus**: The number of cpus to use for the workflow. Default behavior is to use half of the available cpus.
- **clustering**: Which algorithm to use for clustering. Must be one of 'KMedoids', 'Aff_prop'.
- **rescoring**: Which scoring functions to use for rescoring. Must be one or more of 'gnina', 'AD4', 'chemplp', 'rfscorevs', 'LinF9', 'vinardo', 'plp', 'AAScore'.

The software will then create a temporary directory to store the output of the various functions.

In [3]:
software = Path('/home/alacournola/DockM8/software')
protein_file = Path('/home/alacournola/LasB_Hasten_EnamineHTS/6fzx_nowater_noligands_ZNCA_protoss.pdb')
ref_file = Path('/home/alacournola/LasB_Hasten_EnamineHTS/6fzx_ZN_atom.sdf')
pocket = 'reference'
protonation = 'GypsumDL'
docking_library = Path('/home/alacournola/LasB_Hasten_EnamineHTS/vs1_iter1_cpu1.sdf')
docking_programs = ['GNINA', 'SMINA', 'PLANTS']
clustering_metrics = ['bestpose']
clustering_method = 'KMedoids'
rescoring = ['gnina', 'chemplp', 'rfscorevs', 'LinF9', 'RTMScore', 'vinardo', 'KORPL', 'ConvexPLR']
id_column = 'ID'
n_poses = 10
exhaustiveness = 8
ncpus = int(os.cpu_count()-2)
#Create a temporary folder for all further calculations
w_dir = Path(protein_file).parent
print('The working directory has been set to:', w_dir)
(w_dir/'temp').mkdir(exist_ok=True)

The working directory has been set to: /home/alacournola/LasB_Hasten_EnamineHTS


**Pocket Extraction**  

This cell will extract the pocket based on the method specified in the 'pocket' variable. Using 'reference' will use the reference ligand to define the pocket. Using 'dogsitescore' will query the dogsitescorer server and use the pocket with the largest volume.

In [ ]:
if pocket == 'reference':
    pocket_definition = get_pocket(ref_file, protein_file, 15)
    print(pocket_definition)
if pocket == 'RoG':
    pocket_definition = get_pocket_RoG(ref_file, protein_file)
    print(pocket_definition)
elif pocket == 'dogsitescorer':
    pocket_definition = binding_site_coordinates_dogsitescorer(protein_file, w_dir, method='volume')

**Library preparation**  

In [ ]:
if os.path.isfile(w_dir / 'temp' / 'final_library.sdf') == False:
    prepare_library(docking_library, id_column, protonation, ncpus)

**Docking**

This cell will dock all compounds in the receptor, using the reference ligand as a way to define the binding site. (Note: DogSiteScorer not yet implemented here).

The docking algorithms specified in the 'docking' variable will be used.

The docking will be done in on parallel CPU cores depending on the value or the 'parallel' variable.

The docking results are written to the temporary folder. 

In [ ]:
docking(w_dir, protein_file, pocket_definition, software, docking_programs, exhaustiveness, n_poses, ncpus)

All poses are then loaded into memory for clustering

In [ ]:
print('Loading all poses SDF file...')
tic = time.perf_counter()
concat_all_poses(w_dir, docking_programs)
all_poses = PandasTools.LoadSDF(str(w_dir / 'temp' / 'allposes.sdf'), idName='Pose ID', molColName='Molecule', includeFingerprints=False, strictParsing=True)
toc = time.perf_counter()
print(f'Finished loading all poses SDF in {toc-tic:0.4f}!...')

**Clustering**

This cell will perform the clustering according to the values of the 'clusering_metrics', 'clustering_method' and 'parallel' variables. If it detects that the clustering file for that metric has already been generated, it will skip it.

In [ ]:
for metric in clustering_metrics:
        if os.path.isfile(w_dir / 'temp' / f'clustering/{metric}_clustered.sdf') == False:
            cluster_pebble(metric, 'KMedoids', w_dir, protein_file, all_poses, ncpus)

**Rescoring**

This cell will rescore all the clustered .sdf files according to the specified scoring functions and the value of the 'parallel' variable.

In [ ]:
for metric in clustering_metrics:
        rescore_all(w_dir, protein_file, pocket_definition, software, str(w_dir / 'temp' / f'clustering/{metric}_clustered.sdf'), rescoring, ncpus)

**Final ranking methods**

This code calculates the final ranking of compounds using various methods.
- *Method 1* : Calculates ECR value for each cluster center, then outputs the top ranked center.
- *Method 2* : Calculates ECR value for each cluster center, then outputs the average ECR value for each compound.
- *Method 3* : Calculates the average rank of each compound, then ouputs the corresponding ECR value for each compound.
- *Method 4* : Calculates the Rank by Rank consensus
- *Method 5* : Calculates the Rank by Vote consensus
- *Method 6* : Calculates Z-score for each cluster center, then ouputs the top ranked center.
- *Method 7* : Calculates Z-score for each cluster center, then ouputs the average Z-score for each compound.

All methods are then combined into a single dataframe for comparison purposes.

In [15]:
def standardize_scores_scaled(df):
    with open('/home/alacournola/DockM8/scripts/rescoring_functions.json', 'r') as json_file:
        rescoring_functions = json.load(json_file)

    def min_max_standardization(score, min_value, max_value):
        standardized_scores = (score - min_value) / (max_value - min_value)
        return standardized_scores

    for col in df.columns:
        if col != 'Pose ID':
            # Get the min and max values from the JSON file
            column_info = rescoring_functions.get(col)
            if column_info:
                col_min = column_info['parameters']['min']
                col_max = column_info['parameters']['max']
                df[col] = pd.to_numeric(df[col], errors='coerce')
                df[col] = min_max_standardization(df[col], col_min, col_max)
    return df

rescored_dataframe = pd.read_csv(Path(w_dir) / 'temp' / 'rescoring_bestpose_clustered' / 'allposes_rescored.csv')
standardized_dataframe = standardize_scores_scaled(rescored_dataframe)

# Calculate the average and sum for each row, starting from column 2 (third column)
standardized_dataframe['Row Mean'] = standardized_dataframe.iloc[:, 1:].mean(axis=1)
standardized_dataframe['Row Sum'] = standardized_dataframe.iloc[:, 1:].sum(axis=1)
display(standardized_dataframe)

numeric_columns = standardized_dataframe.select_dtypes(include='number')

# Calculate the correlation matrix
correlation_matrix = numeric_columns.corr()

# Display the correlation matrix
display(correlation_matrix)


,Pose ID,RFScoreVS,KORPL,CHEMPLP,ConvexPLR,GNINA_Affinity,CNN-Score,CNN-Affinity,Vinardo,LinF9,RTMScore,Row Mean,Row Sum
0,1000140_GNINA_1,0.197409,0.247100,0.714716,0.8395,0.545013,0.749879,0.315650,0.955014,0.530947,0.225074,0.532030,5.852332
1,1000140_PLANTS_1,0.197929,0.193725,0.726196,0.8105,0.538965,0.282158,0.282180,0.948152,0.537487,0.142952,0.466025,5.126270
2,1000140_SMINA_1,0.195493,0.189333,0.711405,0.8280,0.542527,0.511022,0.290448,0.949806,0.533903,0.163015,0.491495,5.406448
3,1000167_GNINA_1,0.194424,0.182133,0.698852,0.8135,0.537473,0.760323,0.286120,0.943107,0.531273,0.121688,0.506889,5.575781
4,1000167_PLANTS_1,0.203258,0.174125,0.747456,0.8095,0.538015,0.588690,0.277680,0.949985,0.532776,0.122902,0.494439,5.438826
...,...,...,...,...,...,...,...,...,...,...,...,...,...
29349,99990_PLANTS_1,0.193214,0.188183,0.724437,0.8095,0.538291,0.760259,0.272538,0.946313,0.529993,0.137439,0.510017,5.610183
29350,99990_SMINA_1,0.195100,0.199258,0.672444,0.8090,0.543807,0.792944,0.285523,0.949039,0.524909,0.191014,0.516304,5.679343
29351,99993_GNINA_1,0.188214,0.152633,0.691793,0.8185,0.536632,0.846476,0.260863,0.944712,0.529420,0.129887,0.509913,5.609044
29352,99993_PLANTS_1,0.195470,0.159967,0.718366,0.8035,0.529269,0.753219,0.253027,0.938186,0.524883,0.147804,0.502369,5.526062


,RFScoreVS,KORPL,CHEMPLP,ConvexPLR,GNINA_Affinity,CNN-Score,CNN-Affinity,Vinardo,LinF9,RTMScore,Row Mean,Row Sum
RFScoreVS,1.000000,0.193944,0.328923,0.324471,0.347810,-0.151813,0.266621,0.337408,0.224772,0.399394,0.124640,0.124640
KORPL,0.193944,1.000000,0.146821,0.328038,0.318506,-0.060222,0.187434,0.266338,0.054109,0.549520,0.294812,0.294812
CHEMPLP,0.328923,0.146821,1.000000,0.407162,0.411217,-0.246253,0.275348,0.445789,0.363300,0.460380,0.155652,0.155652
ConvexPLR,0.324471,0.328038,0.407162,1.000000,0.802927,0.001485,0.638324,0.864793,0.438483,0.672799,0.456192,0.456192
GNINA_Affinity,0.347810,0.318506,0.411217,0.802927,1.000000,-0.063634,0.659445,0.903723,0.572389,0.657751,0.391931,0.391931
CNN-Score,-0.151813,-0.060222,-0.246253,0.001485,-0.063634,1.000000,0.180575,-0.006375,-0.079448,-0.233663,0.823119,0.823119
CNN-Affinity,0.266621,0.187434,0.275348,0.638324,0.659445,0.180575,1.000000,0.670170,0.519780,0.515353,0.548940,0.548940
Vinardo,0.337408,0.266338,0.445789,0.864793,0.903723,-0.006375,0.670170,1.000000,0.498220,0.603023,0.434091,0.434091
LinF9,0.224772,0.054109,0.363300,0.438483,0.572389,-0.079448,0.519780,0.498220,1.000000,0.299819,0.203866,0.203866
RTMScore,0.399394,0.549520,0.460380,0.672799,0.657751,-0.233663,0.515353,0.603023,0.299819,1.000000,0.302625,0.302625
